In [1]:
def MALA_adaptivevariance(Xzero,Dt,d,N):
    """
    The MALA algorithm with adaptive stepsize to achieve optimal acceptance probability 0.576. 
    The algorithm is implemented for a probability distribution of an isotropic Gaussian random vector.
    It takes as input: initial condition Xzero, initial step size Dt, Markov Chain dimension d, and number of samples N.
    It outputs the average acceptance probability alpha and the average proposal step size.
    """

    X = np.zeros((N,d))  
    X[0,:] = Xzero       
    alpham = []          # Will store the acceptance probabilities in here
    Dtprop = np.zeros(N) # Will store the proposal variances in here
    Dtprop[0] = Dt       
    
    for j in range(N-1): 
        Xprop = X[j,:]-0.5*Dt*X[j,:]+np.sqrt(Dt)*np.random.randn(d)
        qX = np.exp(-(1/(4*0.5*Dt))*np.dot((X[j,:]+(0.5*Dt-1)*Xprop).T,(X[j,:]+(0.5*Dt-1)*Xprop)))
        qprop = np.exp(-(1/(4*0.5*Dt))*np.dot((Xprop+(0.5*Dt-1)*X[j,:]).T,(Xprop+(0.5*Dt-1)*X[j,:])))
        alpha = np.min([1,np.exp(0.5*(np.sum(X[j,:]**2)-np.sum(Xprop**2)))*(qX/qprop)])
        
        # Decide whether or not to accept
        r = np.random.rand(1)  
        if r < alpha:          
            X[j+1,:] = Xprop       # Accepted the proposal! 
        else:                  
            X[j+1,:] = X[j,:]      # Did not accept. X_(j+1) = X_j
        
        alpham += [[alpha]] # Store acceptance probability for this iteration
        
        Dt += (Dt/(j+1))*(alpha-0.576)   # Update the proposal variance using the current acceptance probability and the desired one
        Dtprop[j+1] = Dt                  # Store proposal variance for this iteration
           
    return np.mean(alpham),np.mean(Dtprop)